## Investigation of Chroma DB

**Credit to ChromaDB usage guide**:  This Investigation was built using ChromaDB user guide.  Much of the content (description and code) is copied directly from the [Chroma Usage Guide](https://docs.trychroma.com/guides).  This investigation allows me follow a good guide and make modifications and test functionality as needed.

- https://docs.trychroma.com/guides
- https://docs.trychroma.com/deployment/auth#static-api-token-authentication

### Initiating a persistent Chroma Client

In [1]:
import chromadb
import os

In [2]:
db_path = '/mnt/c/ML/DU/local_rag_llm/prototype/jeff/3-vectorDB/db'

if not os.path.exists(db_path):
    os.makedirs(db_path)

# os.environ['STORAGE_PATH'] = db_path

In [3]:
client = chromadb.PersistentClient(path=db_path)

The client object has a few useful convenience methods

In [4]:
# returns a nanosecond heartbeat. Useful for making sure the client remains connected.
client.heartbeat()

1721350722738373120

In [7]:
# Empties and completely resets the database. WARNING This is destructive and not reversible.

#client.reset() 

## Running Chroma in client/server mode

### prerequisites 

Note: instead of running in python, we will use a docker container.  Run the following docker command.

- expose the server on port `8200`

`docker run -d --name chromadb -v C:/ML/DU/local_rag_llm/db:/chroma/chroma -p 8200:8000 -e IS_PERSISTENT=TRUE -e ANONYMIZED_TELEMETRY=TRUE chromadb/chroma:latest`

Once running, access the chromaDB API documentation at:  http://localhost:8200

**Note:** Chroma also provides an async HTTP client.  For more details: https://docs.trychroma.com/guides

### Docker options:

You can also create a `.chroma_env` file setting the required environment variables and pass it to the Docker container with the `--env-file` flag when running the container.  This will be useful when adding authentication.

`docker run -d --name chromadb -v C:/ML/DU/local_rag_llm/db:/chroma/chroma -p 8200:8000 --env-file ./.chroma_env chromadb/chroma:latest`

where `.chroma_env` file contains:
- IS_PERSISTENT=TRUE
- ANONYMIZED_TELEMETRY=TRUE

docker flags
- `-v` mounts local directory `C:/ML/DU/local_rag_llm/db` to container directory `/chroma/chroma`
- `-p` exposes container port `8000` to localhost port `8200`
- `-d` runs container disconnected (returns to terminal prompt)
- `name` defines the name for the container.  when omitted docker assigns a random name

### Using the Python HTTP-only client

If you are running Chroma in client-server mode, you may not need the full Chroma library. Instead, you can use the lightweight client-only library. In this case, you can install the chromadb-client package. This package is a lightweight HTTP client for the server with a minimal dependency footprint.

`pip install chromadb-client` instead of `pip intall chromadb`

In [8]:
# Chroma's API will run in client-server mode with just this change.

# NOTE: Requires server to be running on port 8200 before running this command.

chroma_client = chromadb.HttpClient(host='localhost', port=8200)

In [9]:
# Try the heartbeat for the server.
chroma_client.heartbeat()

1721351225594153049

## Using Collections

Chroma lets you manage collections of embeddings, using the collection primitive.

Chroma uses collection names in the url, so there are a few restrictions on naming them:

- The length of the name must be between 3 and 63 characters.
- The name must start and end with a lowercase letter or a digit, and it can contain dots, dashes, and underscores in between.
- The name must not contain two consecutive dots.
- The name must not be a valid IP address.

Chroma collections are created with a `name` and an optional `embedding function`. If you supply an embedding function, you must supply it every time you get the collection.

### Embeddings

The embedding function takes text as input, and performs tokenization and embedding. If no embedding function is supplied, Chroma will use sentence transformer `all-MiniLM-L6-v2 model` to create embeddings. This embedding model can create sentence and document embeddings that can be used for a wide variety of tasks. This embedding function runs locally on your machine, and may require you download the model files (this will happen automatically).

See: https://docs.trychroma.com/guides/embeddings to use other sentence transformer embeddings or create a custom embedding.

Embedding functions can be linked to a collection and used whenever you call `add`, `update`, `upsert` or `query`. You can also use them directly which can be handy for debugging.

In [11]:
from chromadb.utils import embedding_functions

# Use a different sentence transformer: all-mpnet-base-v2
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")

In [15]:
# Create a collection
#client.delete_collection(name="my_collection")
collection = chroma_client.create_collection(name="my_collection", embedding_function=sentence_transformer_ef)


In [14]:
# Get a collection
collection = chroma_client.get_collection(name="my_collection", embedding_function=sentence_transformer_ef)

### Collections have a some useful methods

Existing collections can be retrieved by name with `.get_collection`, and deleted with `.delete_collection`. You can also use `.get_or_create_collection` to get a collection if it exists, or create it if it doesn't.


In [ ]:
collection = chroma_client.get_or_create_collection(name="test") # Get a collection object from an existing collection, by name. If it doesn't exist, create it.
collection = chroma_client.get_collection(name="test") # Get a collection object from an existing collection, by name. Will raise an exception if it's not found.
#chroma_client.delete_collection(name="my_collection") # Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible

Other useful Collection methods

In [25]:
# See this used at the last two cells of this notebook
col_list = collection.peek() # returns a list of the first 10 items in the collection
col_num = collection.count() # returns the number of items in the collection
# collection.modify(name="new_name") # Rename the collection

**Existing collections** can be retrieved by name with `.get_collection`, and deleted with `.delete_collection`. You can also use `.get_or_create_collection` to get a collection if it exists, or create it if it doesn't.



In [12]:
# Retrieve collection
collection_A = chroma_client.get_or_create_collection(name="test") # Get a collection object from an existing collection, by name. If it doesn't exist, create it.
collection_A = chroma_client.get_collection(name="test") # Get a collection object from an existing collection, by name. Will raise an exception if it's not found.
#chroma_client.delete_collection(name="my_collection") # Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible


## Test

- Read in a list of text document
- Add them to the collection
- Check the collection has 6 entries
- Query the collection for 2 best fit documents
- Retrieve those documents
- Print the results and note the structure

In [ ]:
# If you are repeating this test, the first reset the database by uncommenting the following lines

#chroma_client.delete_collection(name="my_collection", embedding_function=sentence_transformer_ef)
#chroma_client.create_collection(name="my_collection", embedding_function=sentence_transformer_ef)


## read list of documents from directory

- Read a list of text documents

In [16]:
# read list of .txt files from directory
files = []
for x in os.listdir('data'):
    if x.endswith(".txt"):
        files.append(x)

files

['applying RL to build a binanace trading bot.txt',
 'reinforcement learning and introduction part 1.txt',
 'reinforcement learning and introduction part 2.txt',
 'reinforcement learning and introduction part 3.txt',
 'reinforcement learning and introduction part 4.txt',
 'reinforcement learning DQN part 1.txt']

In [18]:
# copy files contents to a list
content_list = []   # file contents
id_list = []        # IDs, text of your choice

for i, file in enumerate(files):
    with open('data/'+file, "r") as f:
        content_list.append( f.read() )
    id_list.append( 'id' + str(i))

In [19]:
content_list

['Our objective is to make a trading bot that trade cryptocurrency using state-of the-art reinforcment learning. To create our RL agents we will use the following technologies:\n\nPython\nReinforcment Learning\nOPenAI gym\nBinance\nYou don’t need any background in ML to understand the following articles, knowledge of Python will be enough. However, if a part is unclear do not hesitate to contact me.\n\nHere are the different parts of the creation of our bot, each part will be one article:\n\nWe will use Binance data to generate our data that we will customize for our need. All you need is a Binance account, you can create one by clicking here. It is free and one of best trading platform to find cryptocurrency data.\n\nAll of the code for this article will be available on my GitHub.\n\nSteps for generating data:\n\nAdd our Binance API keys\nPulling data from Binance\nStandardize our data as we wish\nSave it in a csv file\nAdd our Binance API keys\nAfter cloning the repository, we will n

In [20]:
id_list

['id0', 'id1', 'id2', 'id3', 'id4', 'id5']

Add Documents to a collection

In [21]:
# Add documents to the collection
collection.add(
    documents=content_list,
    metadatas=[
        {"source": files[0]},   # This holds the file name (source name) for the text being added to the DB
        {"source": files[1]},
        {"source": files[2]},
        {"source": files[3]},
        {"source": files[4]},
        {"source": files[5]}
    ],
    ids=id_list
)

Verify there are 6 items in the collection


In [20]:
print(collection.count())

6


Query the Collection

In [21]:
results = collection.query(
    query_texts=[
        "This is a query about machine learning and data science"
    ],
    n_results=2
)

print(results)

{'ids': [['id5', 'id1']], 'distances': [[1.3659564893812401, 1.4503610804298352]], 'metadatas': [[{'source': 'reinforcement learning DQN part 1.txt'}, {'source': 'reinforcement learning and introduction part 1.txt'}]], 'embeddings': None, 'documents': [['Hi and welcome to the intro series on Reinforcement Learning, today’s topic will be about the DQN algorithm!\n\n\nThis blog post is part of a longer series about Reinforcement Learning (RL). If you are completely unfamiliar with RL, I suggest you read my previous blog posts first.\n\nPreviously we talked about policy gradient algorithms. Today we will have a look at a different family of RL algorithms: Q-learning algorithms. And more specifically, we will focus on the vanilla DQN-algorithm.\n\nThe topics for today’s blog post are:\n\nHistorical significance of DQN\nWhat is Q-Learning?\nDQN Explained\nQ-Learning VS. Policy Gradients\nThere will also be a part 2 for today’s blog post, which will include a basic implementation of DQN.\n\n

Print the result.  Note the data structure is a dictionary

In [22]:
import pprint
pprint.pprint(results)

{'data': None,
 'distances': [[1.3659564893812401, 1.4503610804298352]],
 'documents': [['Hi and welcome to the intro series on Reinforcement Learning, '
                'today’s topic will be about the DQN algorithm!\n'
                '\n'
                '\n'
                'This blog post is part of a longer series about Reinforcement '
                'Learning (RL). If you are completely unfamiliar with RL, I '
                'suggest you read my previous blog posts first.\n'
                '\n'
                'Previously we talked about policy gradient algorithms. Today '
                'we will have a look at a different family of RL algorithms: '
                'Q-learning algorithms. And more specifically, we will focus '
                'on the vanilla DQN-algorithm.\n'
                '\n'
                'The topics for today’s blog post are:\n'
                '\n'
                'Historical significance of DQN\n'
                'What is Q-Learning?\n'
           

Collections have a few useful convenience methods.

In [23]:
col_list = collection.peek() # returns a list of the first 10 items in the collection
col_num = collection.count() # returns the number of items in the collection
# collection.modify(name="new_name") # Rename the collection

In [24]:
print(col_list)
print(f'\n{col_num}')


{'ids': ['id0', 'id1', 'id2', 'id3', 'id4', 'id5'], 'embeddings': [[0.015637988224625587, 0.023393187671899796, -0.018636686727404594, -0.014046601951122284, -0.0020834202878177166, 0.004785358905792236, 0.007279537618160248, -0.035519085824489594, 0.016177749261260033, 0.00990244746208191, 0.02662549912929535, 0.04229314252734184, -0.02552393637597561, 0.13626548647880554, -0.04184751212596893, 0.010002681985497475, -0.0033973052632063627, 0.016156477853655815, 0.029593612998723984, -0.03932398930191994, -0.02277170866727829, -0.03368881717324257, -0.004284380469471216, 0.01080666296184063, -0.023119231685996056, -0.06127709522843361, -0.0683426782488823, -0.015468543395400047, -0.01969340443611145, -0.014431295916438103, 0.03516785055398941, -0.03188183158636093, 0.006375916767865419, 0.08358718454837799, 2.7107550977234496e-06, -0.018341440707445145, -0.0578732043504715, 0.025081884115934372, -0.08243924379348755, -0.056250981986522675, -0.006235369946807623, 0.005906814709305763, 0